# 3_training_mlflow.ipynb

- **Objectif** :
Lancer et suivre les entraînements des modèles ML (Logistic, Random Forest, XGBoost)
avec différentes stratégies de déséquilibre, via MLflow.
---

In [14]:
import os
import mlflow
import mlflow.sklearn
from IPython.display import Markdown as md

## Définir le dossier du projet

In [15]:
# Définir le dossier du projet
os.chdir("..") if "notebooks" in os.getcwd() else None

## Configurer le tracking local MLflow

In [16]:
# Configurer le tracking local MLflow
mlflow.set_tracking_uri("file:./mlruns")
mlflow.set_experiment("Churn_Prediction_Models")


<Experiment: artifact_location='file:///c:/Users/PC/Desktop/sprint5/Churn-Prediction-avec-MLflow/mlruns/667014208243130233', creation_time=1761207273202, experiment_id='667014208243130233', last_update_time=1761207273202, lifecycle_stage='active', name='Churn_Prediction_Models', tags={'mlflow.experimentKind': 'custom_model_development'}>

In [17]:
md("## 🚀 Lancement des entraînements et suivi MLflow")

## 🚀 Lancement des entraînements et suivi MLflow

## 1️⃣ Lancer les scripts

In [18]:
# ==========================================================
# 1️⃣ Lancer les scripts
# ==========================================================
import subprocess

In [19]:
scripts = [
    "src/models/logistic.py",
    "src/models/forest.py",
    "src/models/xgboost.py"
]

In [20]:
for script in scripts:
    print(f"⚙️ Entraînement avec {script} ...")
    subprocess.run(["python", script])
    print(f"✅ Terminé : {script}")

⚙️ Entraînement avec src/models/logistic.py ...
✅ Terminé : src/models/logistic.py
⚙️ Entraînement avec src/models/forest.py ...
✅ Terminé : src/models/forest.py
⚙️ Entraînement avec src/models/xgboost.py ...
✅ Terminé : src/models/xgboost.py


In [21]:
import mlflow

# Récupérer l'ID de l'expérience
experiment = mlflow.get_experiment_by_name("Churn_Prediction_Models")
experiment_id = experiment.experiment_id

# Chercher tous les runs de l'expérience
runs = mlflow.search_runs(experiment_ids=[experiment_id], filter_string="", order_by=["start_time DESC"])
print(runs.columns.tolist())


['run_id', 'experiment_id', 'status', 'artifact_uri', 'start_time', 'end_time', 'metrics.accuracy', 'metrics.f1', 'params.note', 'tags.mlflow.runName', 'tags.mlflow.source.git.commit', 'tags.mlflow.source.type', 'tags.mlflow.user', 'tags.mlflow.source.name', 'tags.model_name', 'tags.imbalance_strategy']


## 2️⃣ Comparaison des résultats

In [22]:
# ==========================================================
# 2️⃣ Comparaison des résultats
# ==========================================================
md("### 📊 Visualisation dans MLflow UI")

### 📊 Visualisation dans MLflow UI

In [23]:
print("""
Pour ouvrir l'interface MLflow :
> mlflow ui --backend-store-uri ./mlruns
Puis ouvrir : http://127.0.0.1:5000
""")


Pour ouvrir l'interface MLflow :
> mlflow ui --backend-store-uri ./mlruns
Puis ouvrir : http://127.0.0.1:5000



In [24]:
md("### 🏆 Sélection du meilleur modèle")

### 🏆 Sélection du meilleur modèle

In [25]:
import pandas as pd

## Charger les runs MLflow

In [27]:
runs_info = client.search_runs(experiment_ids=[experiment.experiment_id])
print(f"✅ {len(runs_info)} runs trouvés")

# Créer le DataFrame
runs_data = []
for run in runs_info:
    run_dict = {
        'run_id': run.info.run_id,
        'tags.model_name': run.data.tags.get('model_name', 'Unknown'),
        'tags.imbalance_strategy': run.data.tags.get('imbalance_strategy', 'Unknown'),
        'metrics.f1_score': run.data.metrics.get('f1_score', 0),
        'metrics.accuracy': run.data.metrics.get('accuracy', 0)
    }
    runs_data.append(run_dict)
runs = pd.DataFrame(runs_data)

# Vérifier les colonnes disponibles
print("📋 Colonnes disponibles :", runs.columns.tolist())


NameError: name 'client' is not defined

In [ ]:
# Vérifier les colonnes disponibles
print("📋 Colonnes disponibles :", runs.columns.tolist())

# Rechercher la colonne correspondant à la métrique F1
f1_cols = [col for col in runs.columns if "f1_score" in col.lower() and col.startswith("metrics.")]

# Si aucune métrique F1 n'est trouvée, afficher un message explicatif au lieu de lever une erreur
if not f1_cols:
    print("⚠️ Aucune métrique F1 trouvée dans les runs MLflow.")
    print("👉 Vérifiez que la métrique F1 a bien été loggée avec `mlflow.log_metric('f1', valeur)`.")
    print("🔎 Exemple de colonnes disponibles liées aux métriques :",
          [col for col in runs.columns if col.startswith("metrics.")])
else:
    # Sélectionner la première colonne F1 trouvée
    f1_col = f1_cols[0]

    # Trier les runs selon la meilleure valeur de F1
    best_run = runs.sort_values(f1_col, ascending=False).iloc[0]

    # Afficher le meilleur modèle avec sa valeur F1
    md(f"**🏆 Meilleur modèle :** {best_run.get('tags.model_name', 'Nom inconnu')} "
       f"({f1_col} = {best_run[f1_col]:.3f})")


📋 Colonnes disponibles : ['run_id', 'experiment_id', 'status', 'artifact_uri', 'start_time', 'end_time', 'metrics.accuracy', 'metrics.f1', 'params.note', 'tags.mlflow.user', 'tags.mlflow.runName', 'tags.mlflow.source.name', 'tags.mlflow.source.type', 'tags.mlflow.source.git.commit', 'tags.model_name', 'tags.imbalance_strategy']


In [ ]:
runs.filter(like="metrics.")


,metrics.accuracy,metrics.f1
0,NaN,NaN
1,0.5,0.5


In [ ]:
best_run = runs.sort_values("metrics.f1_score", ascending=False).iloc[0]
md(f"**Meilleur modèle :** {best_run['tags.model_name']} (F1 = {best_run['metrics.f1']:.3f})")


**Meilleur modèle :** DummyModel (F1 = 0.500)

## Chargement et test du modèle

In [ ]:
# Chargement et test du modèle
model_uri = f"runs:/{best_run['run_id']}/model"
try:
    loaded_model = mlflow.sklearn.load_model(model_uri)
    print("Modèle chargé")
except Exception as e:
    print(f"Erreur chargement modèle: {e}")
    loaded_model = None


In [ ]:
md("### 🔍 Test d'une prédiction avec le modèle chargé")
import joblib
import pandas as pd

## Exemple d’un batch de test

In [ ]:
# Exemple d’un batch de test
X_test = joblib.load("data/X_test.pkl")
print("✅ Prédiction exemple :")
print(loaded_model.predict(X_test[:5]))

✅ Prédiction exemple :
[0 0 0 0 0]
